# Tranformación
Vamos a ver en las 15 hojas del excel las columnas sin nombres y con valores nulos y dropearlas. 

In [5]:
import pandas as pd

# Cargar el archivo de Excel
file_path = 'Internet.xlsx'
excel_data = pd.ExcelFile(file_path)

# Crear un diccionario para almacenar las hojas limpiadas
cleaned_sheets = {}
for sheet_name in excel_data.sheet_names:
    # Cargar cada hoja en un DataFrame
    df = excel_data.parse(sheet_name)
    # Eliminar columnas que contienen solo valores nulos
    df_cleaned = df.dropna(axis=1, how='all')
    # Guardar el DataFrame limpio en el diccionario
    cleaned_sheets[sheet_name] = df_cleaned



Seguimos trabajando con el archivo que exportamos (internet_limpio.xlsx) y vemos la necesidad de limpiar datos nulos y errores. 
1. Revisamos las columnas que contienen errores que aparecen en la hoja "accesos por tecnología": 
    - Columna años se reemplazo los errores por el año 2019, siguiendo la lógica de los datos. 
    - Columna Trimestre se reemplazo los errores por el trimestre 4, siguiendo la lógica de los datos. 
2. De la hoja Acc_vel_Loc_sinrango, Columna Velocidad, se reemplazo errores con el valor 6357120, siguiendo la logica de los datos. 
3. Revisamos la hoja dial_baf, en la columna dial_up y registramos algunos datos nulos correspondientes a dos provincias: formosa y misiones. Por logica de los datos y por ser un volumen insignificante decidí revalorar en 0 los datos vacios/null.
4. Procedo a verificar los tipos de datos en las hojas y columnas reporando () modificaciones.

In [6]:
# Crear un diccionario para almacenar los datos de todas las hojas, conservando las que no requieren modificaciones
sheets_dict = {sheet: excel_data.parse(sheet) for sheet in excel_data.sheet_names}

# Realizar modificaciones solo en las hojas específicas

# Hoja "Accesos Por Tecnología": reemplazar errores con 2019 y en "trimestre" con 4
sheets_dict["Accesos Por Tecnología"] = sheets_dict["Accesos Por Tecnología"].applymap(
    lambda x: 2019 if isinstance(x, str) and x.lower() == 'error' else x)
if 'trimestre' in sheets_dict["Accesos Por Tecnología"].columns:
    sheets_dict["Accesos Por Tecnología"]['trimestre'] = sheets_dict["Accesos Por Tecnología"]['trimestre'].apply(
        lambda x: 4 if isinstance(x, str) and x.lower() == 'error' else x)

# Hoja "Acc_vel_loc_sinrangos": en columna "velocidad" reemplazar errores con 6357120
if 'velocidad' in sheets_dict["Acc_vel_loc_sinrangos"].columns:
    sheets_dict["Acc_vel_loc_sinrangos"]['velocidad'] = sheets_dict["Acc_vel_loc_sinrangos"]['velocidad'].apply(
        lambda x: 6357120 if isinstance(x, str) and x.lower() == 'error' else x)

# Hoja "Dial-BAf": en columna "dial_up" reemplazar valores nulos o vacíos por 0
if 'dial_up' in sheets_dict["Dial-BAf"].columns:
    sheets_dict["Dial-BAf"]['dial_up'] = sheets_dict["Dial-BAf"]['dial_up'].fillna(0)




C:\Users\jerom\AppData\Local\Temp\ipykernel_10960\366169002.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sheets_dict["Accesos Por Tecnología"] = sheets_dict["Accesos Por Tecnología"].applymap(


In [7]:


# Cargar todas las hojas en un diccionario
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Modificar las hojas específicas
# Eliminar asteriscos y convertir a entero en 'Accesos Por Tecnología'
sheets_dict['Accesos Por Tecnología'] = sheets_dict['Accesos Por Tecnología'].replace(r'\*', '', regex=True)
sheets_dict['Accesos Por Tecnología'] = sheets_dict['Accesos Por Tecnología'].apply(pd.to_numeric, errors='ignore')

# Eliminar asteriscos y convertir a entero en 'Acc_vel_loc_sinrangos'
sheets_dict['Acc_vel_loc_sinrangos'] = sheets_dict['Acc_vel_loc_sinrangos'].replace(r'\*', '', regex=True)
sheets_dict['Acc_vel_loc_sinrangos'] = sheets_dict['Acc_vel_loc_sinrangos'].apply(pd.to_numeric, errors='ignore')

# Reemplazar valores nulos por 0 en 'Dial-BAf'
sheets_dict['Dial-BAf'] = sheets_dict['Dial-BAf'].fillna(0)




C:\Users\jerom\AppData\Local\Temp\ipykernel_10960\3900608763.py:7: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  sheets_dict['Accesos Por Tecnología'] = sheets_dict['Accesos Por Tecnología'].apply(pd.to_numeric, errors='ignore')
C:\Users\jerom\AppData\Local\Temp\ipykernel_10960\3900608763.py:11: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  sheets_dict['Acc_vel_loc_sinrangos'] = sheets_dict['Acc_vel_loc_sinrangos'].apply(pd.to_numeric, errors='ignore')


In [ ]:
# PAra una optmización de los recursos y para no ser tan abarcativo decidí eliminar las siguientes tabas: 
# - Dial-BAf y Totales Dial-BAf: Estas hojas contienen datos más específicos sobre conexiones dial-up o tecnologías menos utilizadas, 
# que no son tan representativas en un análisis de conectividad actual.
#  Como mi analisis será sobre la conectividad en la República Argentina, por eso me atendré a las hojas que dicen "sin rango". 

# Definir las hojas a eliminar
sheets_to_drop = ['Dial-BAf', 'Totales Dial-BAf', 'Totales Accesos por rango']

# Cargar solo las hojas que no están en la lista sheets_to_drop
filtered_data = {sheet: excel_data.parse(sheet) for sheet in excel_data.sheet_names if sheet not in sheets_to_drop}

# Guardar el archivo Excel modificado sin las hojas especificadas
output_path = 'Internet_Completo_Filtered.xlsx'
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, df in filtered_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Archivo guardado en:", output_path)

Archivo guardado en: Internet_Completo_Filtered.xlsx
